# Neural network model

In [3]:
%load_ext autoreload
%autoreload 2

%load_ext tensorboard

import datetime
import os
import sys
sys.path.append('..')

import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from matplotlib import pyplot


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
def timestamp():
    return datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
def logName(string=0,dropout=1,lr=0.1,batch=64):
    return string + 'dropout_' + str(dropout) + 'lr' + str(lr) + 'batch' + str(batch) + '__' +timestamp()

Načítanie hodnôt z vopred pripraveného datasetu 1.

In [77]:
testx = np.genfromtxt('./../../data/processed/dataSet1TestX1.csv', delimiter=',')[1:]
testy = np.genfromtxt('./../../data/processed/dataSet1TestY1.csv', delimiter=',')

trainx = np.genfromtxt('./../../data/processed/dataSet1TrainX1.csv', delimiter=',')[1:]
trainy = np.genfromtxt('./../../data/processed/dataSet1TrainY1.csv', delimiter=',')

Načítanie hodnôt z vopred pripraveného datasetu 2.

In [79]:
testx = np.genfromtxt('./../../data/processed/dataSet2TestX.csv', delimiter=',')[1:]
testy = np.genfromtxt('./../../data/processed/dataSet2TestY.csv', delimiter=',')

trainx = np.genfromtxt('./../../data/processed/dataSet2TrainX.csv', delimiter=',')[1:]
trainy = np.genfromtxt('./../../data/processed/dataSet2TrainY.csv', delimiter=',')

Načítanie hodnôt z vopred pripraveného datasetu 3.

In [82]:
testx = np.genfromtxt('./../../data/processed/dataSet3TestX.csv', delimiter=',')[1:]
testy = np.genfromtxt('./../../data/processed/dataSet3TestY.csv', delimiter=',')

trainx = np.genfromtxt('./../../data/processed/dataSet3TrainX.csv', delimiter=',')[1:]
trainy = np.genfromtxt('./../../data/processed/dataSet3TrainY.csv', delimiter=',')

Načítanie hodnôt z vopred pripraveného datasetu 4.

In [5]:
testx = np.genfromtxt('./../../data/processed/dataSet4TestX.csv', delimiter=',')[1:]
testy = np.genfromtxt('./../../data/processed/dataSet4TestY.csv', delimiter=',')

trainx = np.genfromtxt('./../../data/processed/dataSet4TrainX.csv', delimiter=',')[1:]
trainy = np.genfromtxt('./../../data/processed/dataSet4TrainY.csv', delimiter=',')

Načítanie hodnôt z vopred pripraveného datasetu 5.

In [86]:
testx = np.genfromtxt('./../../data/processed/dataSet5TestX.csv', delimiter=',')[1:]
testy = np.genfromtxt('./../../data/processed/dataSet5TestY.csv', delimiter=',')[1:]

trainx = np.genfromtxt('./../../data/processed/dataSet5TrainX.csv', delimiter=',')[1:]
trainy = np.genfromtxt('./../../data/processed/dataSet5TrainY.csv', delimiter=',')[1:]

Definovanie bloku reziduálnej siete.

In [6]:
def dense_residual(X,width):

    X_residuum = X

    X = keras.layers.Dense(width, activation='relu')(X)
    X = keras.layers.Dense(width, activation='relu')(X)
    X = keras.layers.Dense(width, activation='relu')(X)
    X = keras.layers.Dense(width, activation='relu')(X)   
#     X = keras.layers.Dense(width, activation='relu')(X)
    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = keras.layers.Add()([X, X_residuum])
    X = keras.layers.Activation('relu')(X)
   
    """
    X - tensor tvaru (batch,X)
    """ 
#     print(X.shape)
    return X

Definovanie modelu Reziduálnej siete

In [7]:
def Residual_model(input_shape=(25), classes=16):
    
    X_input = keras.Input(input_shape)
    
    X = keras.layers.Dense(256, activation='relu')(X_input)
    X = keras.layers.Dense(128, activation='relu')(X)
    X = dense_residual(X,128)
    
    X = keras.layers.Dropout(0.2)(X) 
    X = keras.layers.Dense(64, activation='relu')(X)
    X = dense_residual(X,64)
    
    X = keras.layers.Dropout(0.2)(X) 
    X = keras.layers.Dense(32, activation='relu')(X)
    X = dense_residual(X,32)

    # output layer
    X = keras.layers.Dense(classes, activation='softmax', name='fc' + str(classes))(X)
    
    # Create model
    model = keras.models.Model(inputs = X_input, outputs = X, name='Residual_model')

    return model

Definovanie Baseline Modelu

In [89]:
def Baseline_model(input_shape=(25), classes=16):

    X_input = keras.Input(input_shape)    
    X = keras.layers.Dense(256, activation='relu')(X_input)
    X = keras.layers.Dense(128, activation='relu')(X)
    X = keras.layers.Dense(64, activation='relu')(X)
#     X = keras.layers.Dropout(0.2)(X) 
    X = keras.layers.Dense(32, activation='relu')(X)

    # output layer
    X = keras.layers.Dense(classes, activation='softmax', name='fc' + str(classes))(X)
    
    
    # Create model
    model = keras.models.Model(inputs = X_input, outputs = X, name='Baseline_model')

    return model

Trénovanie modelu

In [9]:
# model = Baseline_model(input_shape = trainx.shape[1:], classes = 18)
model = Residual_model(input_shape = trainx.shape[1:], classes = 18)

model.compile(loss='sparse_categorical_crossentropy',  
              optimizer=keras.optimizers.SGD(lr=0.005),
#               optimizer=keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False),
#               optimizer=keras.optimizers.RMSprop(learning_rate=0.00001),
              metrics=['sparse_categorical_accuracy'])

history = model.fit(trainx, trainy,
          validation_data=(testx, testy),
          shuffle=True,
          batch_size=128,
          epochs=75,
          callbacks = [keras.callbacks.TensorBoard(
              log_dir=os.path.join("./../../logs", logName('B-5', 'none' , 'RMS-0.00001', 128)),
              histogram_freq=1)],
          verbose=1);

Train on 9013 samples, validate on 2253 samples
Epoch 1/75
9013/9013 [==============================] - 5s 503us/sample - loss: 3.5769 - sparse_categorical_accuracy: 0.0220 - val_loss: 3.2806 - val_sparse_categorical_accuracy: 0.0044
Epoch 2/75
9013/9013 [==============================] - 1s 121us/sample - loss: 3.2692 - sparse_categorical_accuracy: 0.0275 - val_loss: 3.0468 - val_sparse_categorical_accuracy: 0.0173
Epoch 3/75
9013/9013 [==============================] - 1s 101us/sample - loss: 3.0667 - sparse_categorical_accuracy: 0.0457 - val_loss: 2.9033 - val_sparse_categorical_accuracy: 0.0870
Epoch 4/75
9013/9013 [==============================] - 1s 95us/sample - loss: 2.9358 - sparse_categorical_accuracy: 0.0820 - val_loss: 2.8142 - val_sparse_categorical_accuracy: 0.1678
Epoch 5/75
9013/9013 [==============================] - 1s 93us/sample - loss: 2.8520 - sparse_categorical_accuracy: 0.1311 - val_loss: 2.7506 - val_sparse_categorical_accuracy: 0.2175
Epoch 6/75
9013/9013 [==

Epoch 45/75
9013/9013 [==============================] - 1s 111us/sample - loss: 2.2558 - sparse_categorical_accuracy: 0.2203 - val_loss: 2.1998 - val_sparse_categorical_accuracy: 0.2099
Epoch 46/75
9013/9013 [==============================] - 1s 87us/sample - loss: 2.2531 - sparse_categorical_accuracy: 0.2140 - val_loss: 2.1956 - val_sparse_categorical_accuracy: 0.2082
Epoch 47/75
9013/9013 [==============================] - 1s 79us/sample - loss: 2.2505 - sparse_categorical_accuracy: 0.2202 - val_loss: 2.1914 - val_sparse_categorical_accuracy: 0.2104
Epoch 48/75
9013/9013 [==============================] - 1s 89us/sample - loss: 2.2447 - sparse_categorical_accuracy: 0.2262 - val_loss: 2.1871 - val_sparse_categorical_accuracy: 0.2122
Epoch 49/75
9013/9013 [==============================] - 1s 79us/sample - loss: 2.2408 - sparse_categorical_accuracy: 0.2237 - val_loss: 2.1835 - val_sparse_categorical_accuracy: 0.2117
Epoch 50/75
9013/9013 [==============================] - 1s 83us/samp

In [95]:
values = model.predict(testx[:20])

In [96]:
print(np.argmax(values,axis=1))

[3 6 4 5 5 7 7 5 5 3 9 5 5 9 3 5 6 4 9 3]


In [63]:
testy[:20]

array([ 7.,  3.,  5.,  7.,  6.,  4.,  8.,  7.,  8.,  2.,  3.,  7.,  9.,
        9.,  7., 10.,  5.,  7.,  8.,  1.])

In [2]:
 %tensorboard --logdir=./../../logs --bind_all